# Effective countermeasures against a group of interest

This notebook is intended to demonstrate potential uses of SATRAP for the investigation of CTI.

## Preliminaries

The data model of SATRAP is aligned to STIX 2.1, therefore, in the analysis we use the STIX2.1 vocabulary.
We also make use of concepts in the MITRE ATT&CK knowledge base.

The specifications of [ATT&CK in STIX 2.1](https://github.com/mitre-attack/attack-stix-data/blob/master/USAGE.md#the-attck-spec) describe the representation of the following ATT&CK concepts with STIX objects:

- Group: `intrusion-set`
- Mitigation: `course-of-action`
- Technique: `attack-pattern`

## Introduction

The **objective** of this investigation is to identify effective mitigations against techniques used by a specific group of interest.

In this example we assume that the interested organization is a Hospital in Luxembourg. Then, we define the following questions.

### Analysis questions
1. Is the intrusion set "Orangeworm" a potential threat for the organization?
2. In such a case, can we identify relevant courses of action to effectively mitigate the threat?


## Prerequisites
Load MITRE ATT&CK Enterprise using the ETL CLI of SATRAP as explained in the user manual (at `docs/manual/interfaces.md`).

## Preparation
Set up the DB parameters and import the toolkit of SATRAP.

In [ ]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.commons.format_utils import format_dict, tabulate_stix_obj
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)

## Investigation
First, we do a quick search of [Orangeworm in ATT&CK](https://attack.mitre.org/groups/G0071/) to get an overview of the intrusion set. 

Highlights:
- Orangeworm has targeted the healthcare sector in the United States, Europe, and Asia since at least 2015.
- ID of Orangeworm in ATT&CK: `G0071`.

**Answer 1:** We realize that G0071 poses indeed a potential risk for the hospital and thus further investigation is required.

From now on, we can use the ID of Orangeworm to run some automated analysis operations.

### General information of Orangeworm

We start by getting an overview of the group in our CTI SKB.

In [ ]:
try:
	t1 = satrap.search_by_mitre_id("G0071")
	print("Group: G0071")
	for data in t1:
		print(tabulate_stix_obj(data))
except Exception as error:
	print(f"{type(error).__name__}: {error}")

### What techniques does "G0071" use?

To evaluate potential vulnerabilities, we try to get information concerning the techniques used by Orangeworm.

In [ ]:
techniques = satrap.techniques_used_by_groups(["G0071"])
display(techniques)

We already knew about those techniques as they are directly listed in the previous ATT&CK link. So, we try to learn something more by activating the inference engine of SATRAP.

In [ ]:
techniques = satrap.techniques_used_by_groups(["G0071"], infer=True)
display(techniques)

SATRAP has indeed shown a larger list of techniques. And evidently we would like to find out the reason for these techniques to be associated with Orangeworm.

### Analyzing the association of techniques

SATRAP can provide the explanation for all the techniques displayed in one go, however, we decide to go one by one to simplify the analysis and to prioritize the techniques that we consider more likely to be potential threats to our organization.

For instance, we might be intrigued about why the technique $\mathsf{"Windows\ Command\ Shell"}$ forms part of the list above, assuming that the organization's systems run primarily on Windows OS.

In [ ]:
# explain all techniques
# reason = satrap.explain_techniques_used_by_groups(["G0071"])
techniques_explained = satrap.explain_techniques_used_by_groups(["G0071"], "T1059.003")
for reason in techniques_explained.explanations:
	print(format_dict(reason.as_json()), end="\n")

In [ ]:
import resources.ipynb_utils as utils
utils.info_explanation()

As per the note above, the explanation output can be interpreted as follows:
- The application of the inference rule called "transitive-use" resulted in the derivation of data satisfying the statement `{ $use (used:$technique, user:$group); $use isa uses; }` from the underlying query.
- 'Orangeworm uses cmd' and 'cmd uses Windows Command Shell' are relations in the CTI SKB that meet the conditions for the application of "transitive-use".
- The relation 'Orangeworm indirectly-uses Windows Command Shell' is derived as a result of applying the referred rule.


### Mitigating "Windows command shell"

Using the information that Orangeworm uses the technique "Windows Command Shell" via the tool "cmd", we first verify that "cmd" refers to the Windows CLI and then search for mititgations for this technique in SATRAP's knowledge base. To run the concerning SATRAP functions, we obtain the STIX ids from the explanation shown above.

In [ ]:
cmd = satrap.search_by_stix_id("tool--bba595da-b73a-4354-aa6c-224d4de7cb4e")
if cmd:
	print(tabulate_stix_obj(cmd))

In [ ]:
mitigations = satrap.mitigations_for_technique("attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62")
display(mitigations)

At this point, we have identified a suggested course of action (detailed below) to address a specific threat posed by the group that we consider relevant.

**Answer 2 (partial):** 'course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db': 'Execution Prevention'


In [ ]:
mitigation = satrap.search_by_stix_id("course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db")
if mitigation:
	print(tabulate_stix_obj(mitigation))

#### Why is "Domain Groups" listed as a technique used by Orangeworm?

More than one explanation path can lead to the association of a technique with a group, as in the case of $\mathsf{Domain\ Groups}$ for Orangeworm.

In [ ]:
# We take the technique id from the results of "techniques_used_by_groups" above
dg_explanation = satrap.explain_techniques_used_by_groups(["G0071"], "T1069.002")
for tech in dg_explanation.explanations:
	print(format_dict(tech.as_json()), end="\n")

For cases with multiple explanations, the textual explanation can be further clarified using the interactive Graph viewer of TypeDB Studio as explained in the [... section]() of SATRAP's user manual.

We can obtain the query to execute in TypeDB Studio from the explanation result above:

In [ ]:
print(dg_explanation.query)

The next figure shows the explanation paths in TypeDB Studio detailing why "Domain Groups" is a technique used by "G0071"; the relation "indirectly-uses" framed in green corresponds to the inferred relation. These paths correspond to the textual explanations previously displayed.

<p align="center">
	<img src="./resources/figs/tech-g0071-paths.png" alt="Explanation traces in TypeDB Studio" width="85%">
</p>

In this graph, we can clearly see that the conclusion about the intrusion set "Orangeworm" using the attack pattern "Domain Groups" comes from the fact that it relies on the use of both, a tool called "Net" and the malware-family "Kwampirs", each of which in turn make use of "Domain Groups". 

### Which known mitigations are potentially relevant against "G0071"?

Finally, to address more generally the initial analysis question 2, we are interested in identifying a set of mitigations that can be associated with Orangeworm because they address one of more of the techniques used by this group. This can be achieved as follows:

In [ ]:
display(satrap.related_mitigations(group_name="Orangeworm"))

As before, we can get an explanation for why each of the listed mitigations are related to Orangeworm. For instance, we select to get an explanation for "Restrict File and Directory Permissions".

In [ ]:
try:
	rel_explanation = satrap.explain_if_related_mitigation("G0071", "course-of-action--987988f0-cf86-4680-a875-2f6456ab2448")
	print(f"EXECUTED QUERY:\n{rel_explanation.query}\n")
	print("EXPLANATIONS:")
	for tech in rel_explanation.explanations:
		print(format_dict(tech.as_json()), end="\n")
except ValueError as err:
	print(err)

And again, we can run the printed query in TypeDB Studio to visualize an interactive explanation.